# Movie_DataQuest - Search For The Best Single Factor

In [ ]:
# Dependencies
import pandas as pd
import requests
import json
from pprint import pprint
import matplotlib.pyplot as plt
from scipy.stats import linregress
import random
import numpy as np
from config import api_key
from config import tmdb_api_key

### Project description and outline:

Description: How do various movie characteristics affect movie success?

Definition of Success:  Total U.S. Box Office Receipts

Characteristics Considered: 
  - Genre
  - Run time
  - Budget
  - Critical reviews
  - User reviews
  - Awards / nominations
  - MPAA Rating

Hypothesis: The closer box office receipts correlate with the various movie characteristics, 
the stronger the influence of the given characteristic on movie success.



### Steve's Code Goes Here

### Introduction to IMDb Database Preperation   
Movie Data Sets Utilized:  Internet Movie Database (IMDb) & Online Media Database (OMDb).

The IMDb contains a greater range of quantified movie characteristics than does the OMDb.  For 
this reason the IMDb was used to provide the bulk of our testing criteria.  In order to limit 
the scope of the data we pulled only information regarding movies that were feature-length, 
non-"adult", and were released between 2000 and 2019.


In [ ]:
### INITIAL DATA SET ###

# Our first data set is a CSV of film data derived from the data available at https://www.imdb.com/interfaces/. It contains feature-length films released between 2000 and 2019.

# Importing this CSV as a dataframe

imdb_csv = "Resources/trimmed_IMDB_data.csv"
imdb_df = pd.read_csv(imdb_csv, encoding="UTF-8")

imdb_df

In [ ]:
### REFINING DATA SET ###

# Our IMDB data contains hundreds of thousands of movies, many of which had no theatrical release.
# We will combine this data with data constructed from https://movielens.org/, which focuses on released films.

# Importing movielens CSV as a dataframe

movielens_csv = "Resources/movielens_links.csv"
movielens_df =  pd.read_csv(movielens_csv, encoding="UTF-8")

movielens_df

In [ ]:
# Combining the dataframes to find films that are in both

# Renaming themovielens_df "imdbTTID" column to match our imdb_df

movielens_df = movielens_df.rename(columns={"imdbTTID": "tconst"})

# Merging dataframes

big_df = imdb_df.merge(movielens_df, how="inner")

big_df


In [ ]:
### GETTING ADDITIONAL DATA FROM TMDB ###

# Selecting a sample of films for the TMDB API call

# Assigning random number
for index, row in big_df.iterrows():
    random.seed(index)
    big_df.loc[index, "randomNumber"] = random.random()

In [ ]:
# Sorting by random number to randomize
big_df = big_df.sort_values("randomNumber")

# Trimming to 5,000 entries

smaller_df = pd.DataFrame(big_df[:5000])

smaller_df

In [ ]:
# Pulling additional data via a TMDB API call

# Setting base URL
url = "https://api.themoviedb.org/3/movie/"

# Looping through dataframe
for index, row in smaller_df.iterrows():
    movie_id = row["tmdbId"]
    
    full_url = url + str(movie_id) + "?api_key=" + tmdb_api_key
    movie_data = requests.get(full_url).json()

#Setting up error handling and adding data to dataframe

    try:
        smaller_df.loc[index, "Budget"] = movie_data["budget"]
    except:
        smaller_df.loc[index, "Budget"] = "N/A"
        
    try:
        smaller_df.loc[index, "Box Office"] = movie_data["revenue"]
    except:
        smaller_df.loc[index, "Box Office"] = "N/A"
    
smaller_df

In [ ]:
### TRIMMING DATA FOR OMDB API CALLS ###

# Trimming dataframe to only items with Box Office data

boxoffice_df = smaller_df[smaller_df["Box Office"] != "N/A"]
boxoffice_df = boxoffice_df[boxoffice_df["Box Office"] != 0.0]

# Recasting Budget and Box Office as integers

boxoffice_df[['Budget', 'Box Office']] = boxoffice_df[['Budget', 'Box Office']].astype(int)

# Replacing missing values with Nan

boxoffice_df.replace(0, np.nan, inplace=True)

boxoffice_df

In [ ]:
# Reducing to 900 items

sample_df = pd.DataFrame(boxoffice_df[:900])

# Exporting as CSV to preserve data between uses

sample_df.to_csv("Resources/sample.csv",index=False)

In [ ]:
# Importing new dataframe from CSV to preserve data between uses

omdb_csv = "Resources/sample.csv"
omdb_api_df = pd.read_csv(omdb_csv, encoding="UTF-8")

omdb_api_df

#### Conclusion/Segway
The size of the IMDb data set was....   And contained Genre, Run time, Budget, Critical Reviews, User Review, Awards/Nominations and MPAA Ratings.

After the database was acquired and cleaned it was merged with a sample of movies from the OM

### Jim's Code Goes Here

## OMDB Preparation:
    
Although the IMDb does not contain some of the movie characteristics we wished to test, it does contain
the box office information required for our analysis.  We will take a 900 movie sample from the IMDb and
merge it with matching movie information in the OMDb database to form the basis of our analysis.

In [ ]:
movie_name = []
movie_id= ['tt0090605', 'tt1285016']
awards = []
genre = []
metascore =[]
rated = []
ratings = []
runtime = []
imdb_rating = []



In [ ]:
# movie we want to search for.
url = "http://www.omdbapi.com/?i="
api_key_txt = "&apikey=" + api_key

In [ ]:
# Performing a GET request similar to the one we executed
# earlier

for movie in movie_id:
    
    response = requests.get(url + movie +  api_key_txt)
    data = response.json()
    
    awards.append(data['Awards'])
    movie_name.append(data['Title'])
    genre.append(data['Genre'])
    metascore.append(data['Metascore'])
    rated.append(data['Rated'])
    ratings.append(data['Ratings'])
    runtime.append(data['Runtime'])
    imdb_rating.append(data['imdbID'])



In [ ]:
movie_df = pd.DataFrame({'Movie Name' : movie_name, 'Movie ID': movie_id, 'Awards' : awards,
                        'Genre' : genre, 'Metascore' : metascore, 'rated' : rated, 'ratings' : ratings,
                        'Runtime' : runtime, 'IMDB Rating' : imdb_rating } )

In [ ]:
movie_df


#### Conclusion/Segway
The size of the IMDb data set was....   And contained Genre, Run time, Budget, Critical Reviews, User Review, Awards/Nominations and MPAA Ratings.

Once our dataset was established we were ready to run our analysis.

Segway to Box Office vs. Runtime Correlations (Kelly's stuff)

### Kelly's Code Here

## Box Office vs. Runtime Correlations
In completing the box office/runtime correlation we had the following considerations....
and face certain issues in cleaning the data, etc. etc. ...

In [ ]:
test_data_df = pd.DataFrame({"Movie Title": ["Ghostbusters", "Ghostbusters II", "Hocus Pocus"],
                             "Run Time": ["100", "110", "120"],
                             "Box Office Sales": ["200", "250", "300"],
                             "Budget": ["150", "250", "350"]})
test_data_df


In [ ]:
# Correlate Box Office v. Run time


x_values = test_data_df["Box Office Sales"]
y_values = test_data_df["Run Time"]
plt.xlabel("Box Office Sales")
plt.ylabel("Movie Run Time (in minutes)")
plt.title("Movie Run time vs Box Office Sales")
plt.scatter(x_values, y_values, marker = "o", facecolors = "purple", edgecolors="black")

plt.show()


In [ ]:
x_values = test_data_df["Box Office Sales"]
y_values = test_data_df["Run Time"]
plt.xlabel("Box Office Sales")
plt.ylabel("Movie Run Time (in minutes)")
plt.title("Movie Run Time vs Box Office Sales")
plt.scatter(x_values, y_values, marker = "o", facecolors = "purple", edgecolors="black")

(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept

line_eq = "y = " + str(round(slope,2)) + "x +" + str(round(intercept,2))

plt.scatter(x_values, y_values, marker = "o", facecolors="purple", edgecolors="black")
plt.plot(x_values,regress_values,"r-")


plt.xlabel("Box Office Sales")
plt.ylabel("Movie Run Time")
plt.title("Movie Run Time vs Box Office Sales")
plt.annotate(line_eq,(10,15),fontsize=15,color="red")


print(f"The r-value is: {rvalue**2}")


plt.show()

In [ ]:
# Correlate Box Office v. Budget

x_values = test_data_df["Box Office Sales"]
y_values = test_data_df["Budget"]
plt.xlabel("Box Office Sales")
plt.ylabel("Movie Budget Amount")
plt.title("Movie Budget Amount vs Box Office Sales")
plt.scatter(x_values, y_values, marker = "o", facecolors = "green", edgecolors="black")

plt.show()

# Awards / nominations
# MPAA Rating


In [ ]:
x_values = test_data_df["Box Office Sales"]
y_values = test_data_df["Budget"]
plt.xlabel("Box Office Sales")
plt.ylabel("Movie Budget Amount")
plt.title("Movie Budget Amount vs Box Office Sales")
plt.scatter(x_values, y_values, marker = "o", facecolors = "green", edgecolors="black")

(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept

line_eq = "y = " + str(round(slope,2)) + "x +" + str(round(intercept,2))

plt.scatter(x_values, y_values, marker = "o", facecolors="green", edgecolors="black")
plt.plot(x_values,regress_values,"r-")


plt.xlabel("Box Office Sales")
plt.ylabel("Movie Run Time")
plt.title("Movie Run Time vs Box Office Sales")
plt.annotate(line_eq,(10,15),fontsize=15,color="red")


print(f"The r-value is: {rvalue**2}")


plt.show()

#### Conclusion/Segway
Conclusios about MPAA Rating analysis
Box Office vs. Critical Review's (Phillips' stuff)


### Philips' Code Goes Here

## Box Office vs. Critical Reviews 
In completing the box critica reviews we had the following considerations....
and face certain issues in cleaning the data, etc. etc. ...

In [ ]:
# Correlate Box Office v. Critical reviews

In [ ]:
# Correlate Box Office v. User reviews

#### Conclusion/Segway
Conclusios with  with Critical & User reviews analysis
Segway to Awards / Nominations (Jim's Stuff).




### Jim's Correlation Section Goes Here

## Box Office vs. Awards / Nominations
In completing the box office vs. awards / nominations we had the following considerations....
and face certain issues in cleaning the data, etc. etc. ...

In [ ]:
# Correlate Box Office v Awards / nominations

In [ ]:
# option:  Correlate Box Office 

#### Conclusion/Segway
Conclusios with  with Awards/Nominations analysis
Segway linear regression to T-testing final variables.

In [ ]:
# option  Correlate Box Office v MPAA Rating
#genre

## Steve's T-test (Chi Square? ANOVA?)
I looked at movie genres & MPAA ratings.  With genres we look at 
correlations across cataegories and thus performed a different analysis
that linear regression correlations...

In [ ]:
# option: Steve performs t-test across movie genre 
# and Jim does both Awards & MPAA Rating correlations

#### Conclusion/Segway
Conclusions of genre and MPAA analysis
Segway to project conclusion

## Project Conclusion
In completing the box office vs. awards / nominations we had the following considerations....
and face certain issues in cleaning the data, etc. etc. ...